<a href="https://colab.research.google.com/github/Sidhtang/AI-planet-assignment/blob/main/deployment_using_fast_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.7 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
from transformers import AutoTokenizer
import os

# Check if MPS is available and set the device accordingly
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load the model
model_name = "/content/drive/MyDrive/distilbert_general_router_model (1).pth"

# Check if the model file exists and is not empty
if not os.path.exists(model_name) or os.stat(model_name).st_size == 0:
    print(f"Error: Model file not found or empty: {model_name}")
else:
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    try:
        model = torch.load(model_name, map_location=device)
    except RuntimeError as e:
        print(f"Error loading model: {e}")
    else:
        model.eval()  # Put the model in evaluation mode
        model.to(device)

        # Function for inference
        def classify_query(query):
            inputs = tokenizer(query, return_tensors='pt', max_length=256, truncation=True, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            return "Personalization" if prediction == 1 else "Customer_support"

        # Test the model with queries
        queries = [
            "Hey there, you guys got some nice hoodies for me?"
        ]
        for query in queries:
            result = classify_query(query)
            print(f"Query: {query}")
            print(f"Prediction: {result}")
            print()

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-6-09382d3db9af>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

Query: Hey there, you guys got some nice hoodies for me?
Prediction: Personalization



In [7]:
import torch
from transformers import AutoTokenizer
import os
from fastapi import FastAPI
from pydantic import BaseModel

# Check if MPS is available and set the device accordingly
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load the model
model_name = "/content/drive/MyDrive/distilbert_general_router_model (1).pth"

app = FastAPI()

class Query(BaseModel):
    text: str

# Check if the model file exists and is not empty
if not os.path.exists(model_name) or os.stat(model_name).st_size == 0:
    print(f"Error: Model file not found or empty: {model_name}")
else:
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    try:
        model = torch.load(model_name, map_location=device)
    except RuntimeError as e:
        print(f"Error loading model: {e}")
    else:
        model.eval()  # Put the model in evaluation mode
        model.to(device)

        # Function for inference
        def classify_query(query):
            inputs = tokenizer(query, return_tensors='pt', max_length=256, truncation=True, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the correct device
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            return "Personalization" if prediction == 1 else "Customer_support"

        @app.post("/classify")
        async def classify(query: Query):
            result = classify_query(query.text)
            return {"query": query.text, "classification": result}

# To run the FastAPI app in Colab, use the following code:
# from google.colab import output
# output.serve_kernel_port_as_window(8000)
# !pip install nest-asyncio
# import nest_asyncio
# nest_asyncio.apply()
# import uvicorn
# uvicorn.run(app, host="0.0.0.0", port=8000)

Using device: cpu


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-7-ba9a075d87c0>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

In [11]:
from google.colab import output
output.serve_kernel_port_as_window(8000)

import nest_asyncio
nest_asyncio.apply()
import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

INFO:     Started server process [333]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [333]
